# Webserver com Flask

Nesse exercicio iremos implementar um simples webserver com suas funcionalidades e rotas básicas para recebermos requisições de clientes através da rede

In [1]:
import base64
import json
import logging
import socket
import threading
import time

import cv2
import numpy as np
from flask import Flask, request, render_template
from gevent.pywsgi import WSGIServer
import requests
from imutils import paths

### Utilizando a biblioteca logging
A biblioteca logging do python já vem com boas ferramentas para nos auxiliar a monitorar nosso servidor, uma de suas principais funcionalidade são os handlers, com ele nós podemos decedir o que vai ser feito com as mensagens de log, podemos simplesmente printar no terminal, salvar elas em um arquivo, enviar através de http para outro servidor, etc.

In [2]:
log_level = logging.DEBUG
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Handler para printar as mensagens de log no terminal
stream_handler = logging.StreamHandler()
stream_handler.setLevel(log_level)
stream_handler.setFormatter(formatter)

# Handler para salvar as mensagens de log em um arquivo
file_handler = logging.FileHandler("./webserver.log")
file_handler.setLevel(log_level)
file_handler.setFormatter(formatter)

# instancia do Logger que iremos utilizar para escrever as mensagens
logger = logging.getLogger("webserver")
logger.addHandler(stream_handler)
logger.addHandler(file_handler)
logger.setLevel(log_level)


### Instanciando as principais ferramentas do nosso webserver
* Instância do nosso webserver Flask
* Instância do nosso modelo utilizando o opencv
* váriavel __version__ que nos auxilia a monitorar o versionamento que está rodando no servidor

In [3]:
__version__ = "0.0.1"
app = Flask("inference_webserver")
model = cv2.dnn.readNetFromTensorflow("./nose_tracker_best_loss.pb")

Um ponto importante é que iremos trabalhar com um webserver que pode vir a receber requisições em paralelo, contudo temos um único modelo que irá fazer a predição pra todas as requisições. Isso se torna um problema, pois o que ocorre quando no meio da chamada model.forward() executamos o model.setInput() vindo de uma requisição em paralela? Nosso modelo irá nos retornar respostas erradas, podendo ser a resposta de outro input ou até mesmo respostas aleatórias, ou seja, __nosso modelo não é thread safe__.

Para resolver isso vamos utilizar uma instância do Lock disponibilizada pela biblioteca threading. Com o Lock nós conseguimos criar uma trava entre as requisições fazendo com que a requisição só consiga utilizar o modelo quando nenhuma outra requisição esteja utilizando.


In [4]:
lock = threading.Lock()
binary_counter = 0
base64_counter = 0

### Decorators
Nosso servidor Flask utiliza decorators para criar e inserir as funcionalidades das rotas.
Um decorator é um método que nos permite adcionar funcinalidades em uma função/objeto já existente.
A idéia é semelhante a um callback, um decorator criar uma nova função e que dentro desse nova função determiandas tarefas serão executadas e em algum momento essa função criada irá executar o objeto/função de origem.

### Decorator para mensurar o tempo de execução da nossa rota

In [5]:
def time_it(func):
    def decorator(*args, **kwargs):
        started_time = time.time()
        ret = func(*args, **kwargs)
        logger.debug(f"{time.time() - started_time:0.04f} took to be executed.")
        return ret
    decorator.__name__ = func.__name__  # Para funcionar com o Flask
    return decorator

### Criação da nossa rota
Nesse exemplo vamos criar uma rota que irá receber através de um "POST" um json contendo nossa imagem no formato base64

In [6]:

@app.route("/image-64", methods=['POST'])
@time_it
def image_b64():
    print(type(request))
    global base64_counter
    base64_counter += 1
    try:
        json_dict = request.get_json()
        json_dict = request.get_json()
        logger.info(f"Received keys: {json_dict.keys()}")
        if "image" not in json_dict:
                raise RunTimeError("Expected to receive field image")
        image_array = base64.b64decode(json_dict["image"])
        image = cv2.imdecode(np.frombuffer(image_array, dtype=np.uint8), 1)
        batch = cv2.resize(image, (112, 112))
        batch = batch / 255
        batch = batch[np.newaxis]
        batch = np.transpose(batch, [0, 3, 1, 2])
        with lock:
            model.setInput(batch)
            pred = model.forward()[0]
        logger.info(f"pred: {pred}")
        return json.dumps({"x": pred[0], "y": pred[1]}, indent=4, sort_keys=True, default=str), 200, {"Content-Type": "application/json"}
    except Exception as e:
        logger.error(e)

In [7]:
@app.route("/image-bin", methods=['POST'])
@time_it
def image_bin():
    global binary_counter
    binary_counter += 1
    try:
        image = cv2.imdecode(np.frombuffer(request.files["image"].getvalue(), dtype=np.uint8), 1)
        batch = cv2.resize(image, (112, 112))
        batch = batch / 255
        batch = batch[np.newaxis]
        batch = np.transpose(batch, [0, 3, 1, 2])
        with lock:
            model.setInput(batch)
            pred = model.forward()[0]
        logger.info(f"pred: {pred}")
        return json.dumps({"x": pred[0], "y": pred[1]}, indent=4, sort_keys=True, default=str), 200, {"Content-Type": "application/json"}
    except Exception as e:
        logger.error(e)

In [8]:
@app.route("/")
def dashboard():
    try:
        context = {"base64": base64_counter, "binary": binary_counter}
        return render_template("./dashboard.html", **context)
    except Exception as e:
        logger.error(e, exc_info=True)

In [ ]:
LOCALHOST = True
USE_GEVENT = False

if LOCALHOST:
    ip = "127.0.0.1"
else:
    ip = socket.gethostname()
    
if USE_GEVENT:
    server = WSGIServer((ip, 5000), app)
    server.serve_forever()
else:
    app.run(host=ip, port=5000, debug=False)

 * Serving Flask app "inference_webserver" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
2021-04-15 14:04:12,177 - webserver - INFO - Received keys: dict_keys(['image'])
2021-04-15 14:04:12,199 - webserver - INFO - pred: [0.6070819 0.5853746]
2021-04-15 14:04:12,201 - webserver - DEBUG - 0.0419 took to be executed.
127.0.0.1 - - [15/Apr/2021 14:04:12] "POST /image-64 HTTP/1.1" 200 -


<class 'werkzeug.local.LocalProxy'>


2021-04-15 14:04:13,242 - webserver - INFO - Received keys: dict_keys(['image'])
2021-04-15 14:04:13,251 - webserver - INFO - pred: [0.56466174 0.61133623]
2021-04-15 14:04:13,253 - webserver - DEBUG - 0.0219 took to be executed.
127.0.0.1 - - [15/Apr/2021 14:04:13] "POST /image-64 HTTP/1.1" 200 -


<class 'werkzeug.local.LocalProxy'>
